# Getting started with redis

In [331]:
import redis

Redis is simply a key-value whereas values are simply raw bytes. Redis is not a database and does not index. It simply stores for fast retreival.

In [332]:
r = redis.Redis(host="192.168.10.15", port=6379)
r.set("key", "myvalue")
r.get("key")

b'myvalue'

Set up key for testing and run cleanup just in case the script has already been used.

In [333]:
key = "test-1"
r.delete(key)

0

Generate sequential testing messages.

In [334]:
msgs = ["message {}".format(i) for i in range(10)]

Linked list is the most typical use case for redis. No point in storing each suricata alert under separate key, as we would need unique key for each and hashmap lookups are wasteful (and really slow) with large amount of keys.

In [335]:
for msg in msgs:
    r.rpush(key, msg)

In [336]:
r.lrange(key, 0, -1)

[b'message 0',
 b'message 1',
 b'message 2',
 b'message 3',
 b'message 4',
 b'message 5',
 b'message 6',
 b'message 7',
 b'message 8',
 b'message 9']

Items can be pushed to either right or left side of the list. Note that rpush (right-push) essentially appends new items to the end of existing list. Redis *lrange* iterates this list from the left, or beginning. Since new items were appended to end, older items are observed first. Numerical arguments *0* and *-1* signify that iteration should start from beginning and run until last element. But we could use specific offsets if we wanted.

In [337]:
r.lpush(key, "new value in left")
r.rpush(key, "new value in right")

12

Note that we can also push items to *left*, or beginnning, of the list.

In [338]:
r.lrange(key, 0, -1)

[b'new value in left',
 b'message 0',
 b'message 1',
 b'message 2',
 b'message 3',
 b'message 4',
 b'message 5',
 b'message 6',
 b'message 7',
 b'message 8',
 b'message 9',
 b'new value in right']

In [339]:
poppedFromLeft = [r.lpop(key) for pop in range(3)]
poppedFromLeft

[b'new value in left', b'message 0', b'message 1']

In [340]:
poppedFromRight = [r.rpop(key) for pop in range(3)]
poppedFromRight

[b'new value in right', b'message 9', b'message 8']

Note that popped values are removed from the list. So, the list functions as a buffer.

In [341]:
r.lrange(key, 0, -1)

[b'message 2',
 b'message 3',
 b'message 4',
 b'message 5',
 b'message 6',
 b'message 7']

Simple purge would be to delete a key altogether.

In [342]:
r.keys()

[b'test-1', b'suricata1', b'key']

In [343]:
r.delete(key)

1

In [344]:
r.keys()

[b'suricata1', b'key']

Finally, both redis and suricata support channels for simple pub-sub. Iterating over lists or explicitly popping values might not be desireable in streaming scenarios, especially when multiple consumers want access to same stream of data. Ranging over all values is a waste while popping means that one client will essentially *win* and others will never see the message.

In [345]:
channel = r.pubsub()

In [346]:
for i in range(10): 
    r.publish("test", "msg {}".format(i))

In [347]:
channel.subscribe('test')

In [349]:
for i in range(5):
    message = channel.get_message()
    print(message)

None
None
None
None
None
